# Solving MNIST with an MLP

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from fa import *
from train import *
from utils import *

In [ ]:
mnist = mnist_data()
mnist_test = mnist_data(is_train=False)

### First train an ordinary MLP 

In [ ]:
mlp1 = MLP(28*28, flatten(), 10)
loss1 = train_net(mlp1, mnist, 5, regression=False, sample_every=50)['loss']

### Then train an FA MLP

In [ ]:
mlp2 = MLP(28*28, flatten(), 10, linearity=FALinear)
loss2 = train_net(mlp2, mnist, 5, regression=False, sample_every=50)['loss']

### Plot loss curves

In [ ]:
%matplotlib inline

plt.figure(0)
plt.title("training curves")
plt.semilogy(loss1);
plt.semilogy(loss2);
plt.legend(['BP','FA']);

### Measure accuracy

In [ ]:
test_categorical_accuracy(mlp1, mnist_test)

In [ ]:
test_categorical_accuracy(mlp2, mnist_test)